<a href="https://colab.research.google.com/github/satishgunjal/Deep-Learning-Using-Python-Tensorflow-Keras/blob/master/03_Movie_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Classification
* We are going to use 'Word2Vec' technique to convert the movie review(text data) into vector. And use this vector data as input layer in or Neural network
* We are also going to use TensorFlow 2.* version, Keras


In [1]:
import tensorflow as tf
tf.__version__

'2.2.0-rc3'

In [0]:
#Importing other tensorflow libra

import tensorflow_datasets as tfds # to use tensorflow prebuilt datasets
import tensorflow_hub as hub # Contains presaved models for transfer learning

Lets view the list of all the vailable datasets

In [15]:
tfds.list_builders()

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'arc',
 'bair_robot_pushing_small',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'cos_e',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eraser_multi_rc',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flic',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'gr

We are going to use 'imdb_review' dataset

In [16]:
tfds.load('imdb_reviews')

{'test': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'train': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'unsupervised': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>}

Lets split the data into training set, cross validation set and test set. Generally split is 60% for training, 20 for cross validation and 20 % for testing.

Cross validation data is used for fine tuning the model

In [0]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

In [19]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [23]:
# Since our result dataset are not of type list or dictionary we will get data as below
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
print('train_examples_batch= %s' % (train_examples_batch))
print('train_labels_batch= %s' % (train_labels_batch))

train_examples_batch= tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film wa

Above result shows 10 movie review with 10 corresponding labels. 1 = positive review, 0 = negative review

* Now need to convert our movie review text using Word2Vec to vectors so that we can use it in our model
* We are going to use TensroFlow Hub to use pretrained model of Word2Vec 


In [0]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)
# hub_layer is or input layer which converts text inti vectors

Lets try and convert movie review text to vector using hub_layer

In [26]:
train_examples_batch[:1]

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."],
      dtype=object)>

In [25]:
hub_layer(train_examples_batch[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ]],
      dtype=float32)>

So the pretrained Word2Vec model converted the movie review into a vector of shape(1,20). 

## Creating a model

In [28]:
model = tf.keras.Sequential()

model.add(hub_layer) # input layer
model.add(tf.keras.layers.Dense(16, activation = 'relu')) # 16 neurons in hidden layer
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid')) # Since single callss classification using sigmoid function. Use 'softmax' for multclass classification
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Train the model

In [35]:
## Shuffle the samples, total 20 iterations, pass validation data for fine tuning, verbose =1 for printing the output
model.fit(train_data.shuffle(10000).batch(512),  
          epochs=20, 
          validation_data = validation_data.batch(512),
          verbose =1)

Epoch 1/20
30/30 [==============================] - 3s 110ms/step - loss: 0.8737 - accuracy: 0.5424 - val_loss: 0.7430 - val_accuracy: 0.5558
Epoch 2/20
30/30 [==============================] - 3s 104ms/step - loss: 0.7080 - accuracy: 0.5735 - val_loss: 0.6848 - val_accuracy: 0.5920
Epoch 3/20
30/30 [==============================] - 3s 102ms/step - loss: 0.6683 - accuracy: 0.6044 - val_loss: 0.6540 - val_accuracy: 0.6210
Epoch 4/20
30/30 [==============================] - 3s 102ms/step - loss: 0.6360 - accuracy: 0.6402 - val_loss: 0.6281 - val_accuracy: 0.6530
Epoch 5/20
30/30 [==============================] - 3s 105ms/step - loss: 0.6067 - accuracy: 0.6752 - val_loss: 0.6020 - val_accuracy: 0.6834
Epoch 6/20
30/30 [==============================] - 3s 105ms/step - loss: 0.5759 - accuracy: 0.7094 - val_loss: 0.5715 - val_accuracy: 0.7110
Epoch 7/20
30/30 [==============================] - 3s 103ms/step - loss: 0.5394 - accuracy: 0.7446 - val_loss: 0.5385 - val_accuracy: 0.7430
Epoch 

## Testing the model

In [38]:
model.predict([['This the worst movie i have even seen']])

array([[0.08275743]], dtype=float32)

Since result is near to zero means its negative review

In [39]:
model.predict([['This is best movie of all time']])

array([[0.8732744]], dtype=float32)

Since result is near to one its positive review

In [40]:
# We can also predict for multiple review's at a time
model.predict(["This is the worst movie I have ever seen",
              "An excellent movie that I enjoyed a lot",
              "how can one make such a horrible movie? there is no story, acting direction everything is very poor"])

array([[0.15563773],
       [0.9857479 ],
       [0.03307325]], dtype=float32)